#### tensorflow基础API使用

In [1]:
# 使用tf.keras搭建回归模型，数据集使用加利福尼亚的房价预测
#导入必要的库即版本

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
#import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.4
pandas 0.23.4
sklearn 0.19.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


#### 常量

In [2]:
t = tf.constant([[1.,2.,3.],[4,5,6]])

# 索引操作
print(t)
print(t[:,1:])
print(t[...,1])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [3]:
# 算子操作
print(t + 10)
print(tf.square(t))# 平方
print(t @ tf.transpose(t))   #矩阵乘法
# 1.0版本中需要session和run才可以得到具体的值
# 2.0默认eager execution打开，直接可以获得具体的值

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [4]:
# numpy转换
print(t.numpy()) # tf转numpy
print(np.square(t))
np_t = np.array([[1,2,3],[4,5,6]])
print(tf.constant(np_t))# numpy转tf

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


In [5]:
# 0维tfnsor    Scalars
t = tf.constant(2.718)
print(t.numpy())
print(t.shape)

2.718
()


字符串

In [6]:
# strings/单个字符串
t = tf.constant('cafe')
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit='UTF8_CHAR'))
print(tf.strings.unicode_decode(t, 'UTF8'))# 将字符串转换为UTF8编码

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [7]:
# string array 数组字符串
t = tf.constant(['cafe','coffee','咖啡'])
print(tf.strings.length(t, unit='UTF8_CHAR'))# 求数组中每个字符串的长度
r = tf.strings.unicode_decode(t, 'UTF8')
print(r)# 输出为RaggedTensor对象，即不完整的矩阵，第二维各行长度不一致/不规则

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [8]:
# RaggedTensor对象
r = tf.ragged.constant([[11,12],[21,22,23],[],[41]])
# 索引操作
print(r)
print(r[1])
print(r[1:2])

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>


In [9]:
# 拼接操作
r2 = tf.ragged.constant([[51,52],[],[71]])
print(tf.concat([r,r2],axis=0))# 横向拼接

# 纵向拼接需要维度有相同的行数
r3 = tf.ragged.constant([[13,14],[15],[],[42,43]])
print(tf.concat([r,r3],axis=1))

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>
<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [], [41, 42, 43]]>


In [10]:
# raggedTensor转换为普通tensor,对应位置用0补齐,0值在正常值后面
print(r.to_tensor())

W0813 23:06:39.181439  6828 deprecation.py:323] From D:\SoftWare\Anaconda3\lib\site-packages\tensorflow\python\ops\ragged\ragged_tensor.py:1553: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


4、sparse tensor 与tf.Variable

In [17]:
# sparse tensor
# 如果一个矩阵中大部分位置都是0，只有少部分位置是1的话，那就把位置是1的坐标和值记录下来
s = tf.SparseTensor(indices=[[0,1],[1,0],[2,3]],
                   values=[1.,2.,3.],dense_shape=[3,4])
print(s)
print(tf.sparse.to_dense(s))# 转换为普通的tensor

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [20]:
# sparse tensors操作
s2 = s * 2.0   # 乘法
print(s2)
# 加法无法使用
try:
    s3 = s + 1
except TypeError as ex:
    print(ex)
    
s4 = tf.constant([[10.,20],
                 [30,40],
                 [50,60],
                 [70,80]])
print(tf.sparse.sparse_dense_matmul(s, s4))  # 矩阵乘法

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'float'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [27]:
# sparse tensor
# indices必须排好序，未排好序使用reorder函数
s5 = tf.SparseTensor(indices=[[0,2],[0,1],[2,3]],
                   values=[1.,2.,3.],dense_shape=[3,4])
print(s5)
s6 = tf.sparse.reorder(s5)
print(tf.sparse.to_dense(s6))# 转换为普通的tensor

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [28]:
# 变量
v = tf.Variable([[1.,2,3],[4,5,6]])
print(v)
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [30]:
#变量赋值，调用assign函数,不能使用等于号
v.assign(2*v)
print(v.numpy())

v[0,1].assign(42)
print(v.numpy())

v[1].assign([7,8,9.])
print(v.numpy())

try:
    v[1] = [7.,8.,9.]
except TypeError as ex:
    print(ex)

[[ 4. 84. 12.]
 [14. 16. 18.]]
[[ 4. 42. 12.]
 [14. 16. 18.]]
[[ 4. 42. 12.]
 [ 7.  8.  9.]]
'ResourceVariable' object does not support item assignment
